In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nba-games/players.csv
/kaggle/input/nba-games/teams.csv
/kaggle/input/nba-games/ranking.csv
/kaggle/input/nba-games/games_details.csv
/kaggle/input/nba-games/games.csv


# **Objetivo**:

<font size="4">- Conocer si se ganaron más partidos de local o de visitante.</font>

<font size="4">- Qué equipos hicieron la mayor cantidad de puntos, de local y visitante.</font>

<font size="4">- Cuántos partidos ganaron los distintos equipos de local y visitante.</font>

<font size="4">- Cómo se han comportado los puntos anotados a lo largo de los años.</font>

<font size="4">- Contra qué equipos se ha ganado más de local y visitante.</font>

In [2]:
#Cargo los archivos
games = pd.read_csv('/kaggle/input/nba-games/games.csv')
games.head(3)

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,...,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1


In [3]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26651 entries, 0 to 26650
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GAME_DATE_EST     26651 non-null  object 
 1   GAME_ID           26651 non-null  int64  
 2   GAME_STATUS_TEXT  26651 non-null  object 
 3   HOME_TEAM_ID      26651 non-null  int64  
 4   VISITOR_TEAM_ID   26651 non-null  int64  
 5   SEASON            26651 non-null  int64  
 6   TEAM_ID_home      26651 non-null  int64  
 7   PTS_home          26552 non-null  float64
 8   FG_PCT_home       26552 non-null  float64
 9   FT_PCT_home       26552 non-null  float64
 10  FG3_PCT_home      26552 non-null  float64
 11  AST_home          26552 non-null  float64
 12  REB_home          26552 non-null  float64
 13  TEAM_ID_away      26651 non-null  int64  
 14  PTS_away          26552 non-null  float64
 15  FG_PCT_away       26552 non-null  float64
 16  FT_PCT_away       26552 non-null  float6

In [4]:
#Filtro las columnas que voy a utilizar
games = games[['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT','TEAM_ID_home',
              'TEAM_ID_away','PTS_home', 'HOME_TEAM_WINS']]
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,TEAM_ID_home,TEAM_ID_away,PTS_home,HOME_TEAM_WINS
0,2022-12-22,22200477,Final,1610612740,1610612759,126.0,1
1,2022-12-22,22200478,Final,1610612762,1610612764,120.0,1
2,2022-12-21,22200466,Final,1610612739,1610612749,114.0,1
3,2022-12-21,22200467,Final,1610612755,1610612765,113.0,1
4,2022-12-21,22200468,Final,1610612737,1610612741,108.0,0


In [5]:
#Cambio el tipo de dato de la fecha
games['GAME_DATE_EST'] = pd.to_datetime(games['GAME_DATE_EST'], format='%Y-%m-%d')

In [6]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26651 entries, 0 to 26650
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   GAME_DATE_EST     26651 non-null  datetime64[ns]
 1   GAME_ID           26651 non-null  int64         
 2   GAME_STATUS_TEXT  26651 non-null  object        
 3   TEAM_ID_home      26651 non-null  int64         
 4   TEAM_ID_away      26651 non-null  int64         
 5   PTS_home          26552 non-null  float64       
 6   HOME_TEAM_WINS    26651 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 1.4+ MB


In [7]:
#Agrego la columna en la que se va a almacenar el año en que se jugó el partido
games['year'] = games['GAME_DATE_EST'].dt.year
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,TEAM_ID_home,TEAM_ID_away,PTS_home,HOME_TEAM_WINS,year
0,2022-12-22,22200477,Final,1610612740,1610612759,126.0,1,2022
1,2022-12-22,22200478,Final,1610612762,1610612764,120.0,1,2022
2,2022-12-21,22200466,Final,1610612739,1610612749,114.0,1,2022
3,2022-12-21,22200467,Final,1610612755,1610612765,113.0,1,2022
4,2022-12-21,22200468,Final,1610612737,1610612741,108.0,0,2022
